In [62]:
#!/usr/bin/env python3

from PIL import Image
from scipy.io import loadmat
import numpy as np
from pathlib import Path
from tqdm import tqdm
import itertools
from scipy.ndimage.morphology import distance_transform_edt


def generate_dataset(out_dir, amd, control):
    amd_files = Path(amd).glob("*")
    control_files = Path(control).glob("*")
    raw_files = list(itertools.chain(amd_files, control_files))
    Path(out_dir).mkdir()
    img_cnt = 0
    for file_ind in tqdm(range(len(raw_files)), desc="data generation"):
        file = raw_files[file_ind]
        img_dict = loadmat(file)
        layers = img_dict["layerMaps"]
        images = img_dict["images"].transpose((2, 0, 1))
        for image, layer in zip(images, layers):
            x_ind, _ = np.where(~np.isnan(layer))
            if len(x_ind):
                x_min = x_ind.min()
                x_max = x_ind.max()
                width = x_max - x_min
                if width > 64:
                    number_of_slices = width // 64
                    for num in range(number_of_slices):
                        img = image[:, x_min + num * 64: x_min + (num + 1) * 64]
                        lyr = layer[x_min + num * 64: x_min + (num + 1) * 64, :]
                        mask = np.zeros((image.shape[0], 64), dtype="uint8")
                        for col_idx, col in enumerate(lyr):
                            border1 = int(col[0])
                            border2 = int(col[1])
                            border3 = int(col[2])
                            mask[:border1 + 1, col_idx] = 0
                            mask[border1 + 1:border2 + 1, col_idx] = 1
                            mask[border2 + 1:border3 + 1, col_idx] = 2
                            mask[border3 + 1:, col_idx] = 3
                        Image.fromarray(img).save(f"{out_dir}/img_{img_cnt}.png")
                        Image.fromarray(mask_rbr).save(f"{out_dir}/mask_{img_cnt}.png")
                        img_cnt += 1
                        
def show_layers_from_boundary_array(img_array, layer_array):
    err_msg = "layer boundaries not compatible with image width"
    assert img_array.shape[1] == layer_array.shape[0], err_msg
    dme_colorcode = {
        1: (180, 200, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100)
    }
    amd_colorcode = {
        1: (180, 200, 250),
        2: (120, 200, 250),
    }
    zeros = np.zeros_like(img_array, dtype="uint8")
    hue = np.zeros_like(img_array, dtype="uint8")
    saturation = np.zeros_like(img_array, dtype="uint8")
    value = np.zeros_like(img_array, dtype="uint8")
    mask = np.zeros(img_array.shape, dtype="uint8")
    for w in range(img_array.shape[1]):
        if ~np.isnan(layer_array[w, :]).any():
            last_boundary = int(layer_array[w, 0])
            for idx, h in enumerate(layer_array[w, 1:]):
                curr_boundary = int(h) + 1
                mask[last_boundary:curr_boundary, w] = idx + 1
                last_boundary = curr_boundary
    if layer_array.shape[1] == 8:
        for klass, hsv in dme_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    if layer_array.shape[1] == 3:
        for klass, hsv in amd_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    alpha = np.zeros_like(img_array, dtype="uint8")
    alpha[mask != 0] = 255
    img_stack = np.array([zeros, zeros, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    alphaimg = Image.fromarray(alpha)
    img_with_boundaries = Image.composite(colored_mask, img, alphaimg)
    return img_with_boundaries


def boundary_length_distribution(folder):
    lengths = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                lengths.append(x_inds[-1] - x_inds[0])
    return lengths


def boundary_middle_distribution(folder):
    middles = []
    for file in folder:
        data = loadmat(file)
        for bscan in data["layerMaps"]:
            x_inds, _ = np.where(~np.isnan(bscan))
            if x_inds.any():
                middles.append(x_inds[len(x_inds) // 2])
    return middles


def show_boundaries_from_mask_array(img_array, mask_array):
    empty = mask_array.copy()
    ones = np.ones(img_array.shape, dtype="uint8") * 255
    img_stack = np.array([empty, empty, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([empty, ones, ones]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    maskimg = Image.fromarray(mask_array)
    img_with_boundaries = Image.composite(colored_mask, img, maskimg)
    return img_with_boundaries


def show_boundary_from_boundary_array(img_array, layer_array):
    err_msg = "layer boundaries not compatible with image width"
    assert img_array.shape[1] == layer_array.shape[0], err_msg
    mask = np.zeros(img_array.shape, dtype="uint8")
    for w_idx in range(img_array.shape[1]):
        if ~np.isnan(layer_array[w_idx, :]).any():
            for h_idx in layer_array[w_idx, :]:
                mask[int(h_idx), w_idx] = 255
    empty = mask.copy()
    ones = np.ones(img_array.shape, dtype="uint8") * 255
    img_stack = np.array([empty, empty, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([empty, ones, ones]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    maskimg = Image.fromarray(mask)
    img_with_boundaries = Image.composite(colored_mask, img, maskimg)
    return img_with_boundaries

# LOAD FILES

In [11]:
amd = list(Path("../../dataset/raw/AMD/").glob("*"))
control = list(Path("../../dataset/raw/Control/").glob("*"))
dme = list(Path("../../dataset/raw/2015_BOE_Chiu/").glob("*"))

# DME

In [67]:
data = loadmat(dme[0])
idx = 10
img = data["images"][:, :, idx]
lyr = data["automaticLayersDME"][:, :, idx]
# lyr2 = data["manualLayers2"][:, :, idx]
lyr3 = data["automaticLayersNormal"][:, :, idx]

In [65]:
show_boundary_from_boundary_array(img, lyr.T).show()
# show_layers_from_boundary_array(img, lyr3.T).show()

In [110]:
def show_layers_from_boundary_array(img_array, layer_array, fluid=None):
    err_msg = "layer boundaries not compatible with image width"
    assert img_array.shape[1] == layer_array.shape[0], err_msg
    dme_colorcode = {
        1: (170, 160, 250),
        2: (120, 200, 250),
        3: (80, 200, 250),
        4: (50, 230, 250),
        5: (20, 230, 250),
        6: (0, 230, 250),
        7: (0, 230, 100),
        8: (180, 255, 255)  # fluid
    }
    amd_colorcode = {
        1: (180, 200, 250),
        2: (120, 200, 250),
    }
    zeros = np.zeros_like(img_array, dtype="uint8")
    hue = np.zeros_like(img_array, dtype="uint8")
    saturation = np.zeros_like(img_array, dtype="uint8")
    value = np.zeros_like(img_array, dtype="uint8")
    mask = np.zeros(img_array.shape, dtype="uint8")
    for w in range(img_array.shape[1]):
        if ~np.isnan(layer_array[w, :]).any():
            last_boundary = int(layer_array[w, 0])
            for idx, h in enumerate(layer_array[w, 1:]):
                curr_boundary = int(h) + 1
                mask[last_boundary:curr_boundary, w] = idx + 1
                last_boundary = curr_boundary
    if fluid is not None:
        mask[fluid != 0] = 8
    if layer_array.shape[1] == 8:
        for klass, hsv in dme_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    if layer_array.shape[1] == 3:
        for klass, hsv in amd_colorcode.items():
            hue[mask == klass] = hsv[0]
            saturation[mask == klass] = hsv[1]
            value[mask == klass] = hsv[2]
    alpha = np.zeros_like(img_array, dtype="uint8")
    alpha[mask != 0] = 255
    img_stack = np.array([zeros, zeros, img_array]).transpose((1, 2, 0))
    img = Image.fromarray(img_stack, mode="HSV")
    colored_mask_stack = np.array([hue, saturation, value]).transpose((1, 2, 0))
    colored_mask = Image.fromarray(colored_mask_stack, mode="HSV")
    alphaimg = Image.fromarray(alpha)
    img_with_boundaries = Image.composite(colored_mask, img, alphaimg)
    return img_with_boundaries

In [91]:
test = data["manualFluid1"][..., 40]
test[test != 0] = 255
test = np.asarray(test, dtype="uint8")
Image.fromarray(test).show()

In [117]:
idx = 45
img = data["images"][..., idx]
lyr = data["automaticLayersDME"][..., idx]
fluid = data["manualFluid1"][..., idx]
show_layers_from_boundary_array(img, lyr.T, fluid).show()

# AMD

In [41]:
data = loadmat(amd[0])
idx = 30
img = data["images"][:, :, idx]
lyr = data["layerMaps"][idx]

In [59]:
show_layers_from_boundary_array(img, lyr).show()
show_boundary_from_boundary_array(img, lyr).show()